In [57]:
import cv2
import numpy as np


In [58]:
original_lena = cv2.imread('lena.bmp', 0)

# Convolution function

This function is the same as the previous homework 9.

In [59]:
def convolution(neighborhood, convolution_mask):
    value = 0
    for i in range(neighborhood.shape[0]):
        for j in range(neighborhood.shape[1]):
            value += (neighborhood[i, j] * convolution_mask[convolution_mask.shape[0] - i - 1, convolution_mask.shape[1] - j - 1])
    return value

# Convolution applied to the whole image

In [60]:
def convolution_whole_image(img, convolution_mask):
    convolution_result = np.zeros((img.shape[0] - convolution_mask.shape[0] + 1, img.shape[1] - convolution_mask.shape[1] + 1))
    for i in range(convolution_result.shape[0]):
        for j in range(convolution_result.shape[1]):
            convolution_result[i, j] = convolution(img[i:i+convolution_mask.shape[0], j:j+convolution_mask.shape[1]], convolution_mask)
    return convolution_result

# Convolution result to Laplacian output

This function converts the convolution result into a Laplacian output array.
The definition is as in the ppt, for example:

Input pixel gradient magnitude >= threshold (15) → Laplacian output pixel t = 1  
Input pixel gradient magnitude <= -threshold (15) → Laplacian output pixel t = -1  
Else → Laplacian output pixel t = 0  

In [61]:
def convolution_result_to_Laplacian_output(convolution_result, threshold):
    laplacian_output = np.zeros(convolution_result.shape)
    for i in range(convolution_result.shape[0]):
        for j in range(convolution_result.shape[1]):
            if convolution_result[i,j] > 0 and convolution_result[i,j] >= threshold:
                laplacian_output[i,j] = 1
            elif convolution_result[i,j] < 0 and convolution_result[i,j] <= -threshold:
                laplacian_output[i,j] = -1
            else:
                laplacian_output[i,j] = 0
    return laplacian_output

# Apply zero-crossing on Laplacian output

If the Laplacian output pixel is:
1. $t = 1$, and one of its 8 neighbors is -1, then the result image pixel is 0.
2. $t = -1$ or $t = 0$, then the result image pixel is 255.

## Implementation details:

`line 2`: First we initialize the result image to be a 255-filled array, so that we only change the pixel values when the above first condition is satisfied.

`line 3,4`: Then we loop though each pixel

`line 5`: if the Laplacian output pixel is 1, then we check its 8 neighbors to see if the above first condition is satisfied.

`line 6`: a flag `has_negative_neighbor` is initialized to be `False`, it is modified when a negative neighbor is found.

`line 7-9`: we loop through the 8 neighbors, as each of the neighbor's index is calculated by adding `di` to the current row, and `dj` to the current column.

`line 10-12`: if the resulting index of a neighbor is within the image, and its value is -1

`line 13`: then we set `has_negative_neighbor` to be `True`, and break the loop, so we skp rest of the columns in the current row

`line 15,16`: we also don't need to check the rest of the rows if a negative neighbor is found.

`line 17,18`: finally, set the pixel to be 0 if a negative neighbor is found.

In [62]:
def Laplacian_output_to_result_image(Laplacian_output):
    result_image = np.full(Laplacian_output.shape, 255)
    for i in range(Laplacian_output.shape[0]):
        for j in range(Laplacian_output.shape[1]):
            if Laplacian_output[i,j] == 1:
                has_negative_neighbor = False
                for di in range(-1, 2):                    
                    for dj in range(-1, 2):                
                        ni, nj = i+di, j+dj
                        if (0 <= ni < Laplacian_output.shape[0] and 
                            0 <= nj < Laplacian_output.shape[1] and 
                            Laplacian_output[ni, nj] == -1):
                            has_negative_neighbor = True
                            break                          
                    if has_negative_neighbor:
                        break                             
                if has_negative_neighbor:
                    result_image[i,j] = 0
    return result_image


# (a) Laplace Mask1: 15

<img src = "ppt_info_images/Laplacian_mask1.png" width = 300/>

In [63]:
def Laplace_Mask1(img, threshold):
    kernel = np.array([
        [0, 1, 0],
        [1, -4, 1],
        [0, 1, 0]
    ])
    convolution_result = convolution_whole_image(img, kernel)
    Laplacian_output = convolution_result_to_Laplacian_output(convolution_result, threshold)
    return Laplacian_output_to_result_image(Laplacian_output)

In [64]:
Laplace_Mask1_img = Laplace_Mask1(original_lena, 15)
cv2.imwrite('result_images/Laplace_Mask1_15.bmp', Laplace_Mask1_img)


True

# (b) Laplace Mask2: 15

<img src = "ppt_info_images/Laplacian_mask2.png" width = 300/>

In [65]:
def Laplace_Mask2(img, threshold):
    kernel = np.array([
        [1, 1, 1],
        [1, -8, 1],
        [1, 1, 1]
    ]) / 3
    convolution_result = convolution_whole_image(img, kernel)
    Laplacian_output = convolution_result_to_Laplacian_output(convolution_result, threshold)
    return Laplacian_output_to_result_image(Laplacian_output)



In [66]:
Laplace_Mask2_img = Laplace_Mask2(original_lena, 15)
cv2.imwrite('result_images/Laplace_Mask2_15.bmp', Laplace_Mask2_img)

True

# (c) Minimum variance Laplacian: 20

<img src = "ppt_info_images/minimum_var_Laplacian.png" width = 300/>

In [67]:
def Minimum_Variance_Laplacian(img, threshold):
    kernel = np.array([
            [2., -1, 2],
            [-1, -4, -1],
            [2, -1, 2]
        ]) / 3
    convolution_result = convolution_whole_image(img, kernel)
    Laplacian_output = convolution_result_to_Laplacian_output(convolution_result, threshold)
    return Laplacian_output_to_result_image(Laplacian_output)


In [68]:
Minimum_Variance_Laplacian_img = Minimum_Variance_Laplacian(original_lena, 20)
cv2.imwrite('result_images/Minimum_Variance_Laplacian_20.bmp', Minimum_Variance_Laplacian_img)

True

# (d) Laplace of Gaussian: 3000

<img src = "ppt_info_images/Laplacian_of_Gaussian.png" width = 300/>

In [69]:
def Laplace_of_Gaussian(img, threshold):
    kernel = np.array([
            [0, 0, 0, -1, -1, -2, -1, -1, 0, 0, 0],
            [0, 0, -2, -4, -8, -9, -8, -4, -2, 0, 0],
            [0, -2, -7, -15, -22, -23, -22, -15, -7, -2, 0],
            [-1, -4, -15, -24, -14, -1, -14, -24, -15, -4, -1],
            [-1, -8, -22, -14, 52, 103, 52, -14, -22, -8, -1],
            [-2, -9, -23, -1, 103, 178, 103, -1, -23, -9, -2],
            [-1, -8, -22, -14, 52, 103, 52, -14, -22, -8, -1],
            [-1, -4, -15, -24, -14, -1, -14, -24, -15, -4, -1],
            [0, -2, -7, -15, -22, -23, -22, -15, -7, -2, 0],
            [0, 0, -2, -4, -8, -9, -8, -4, -2, 0, 0],
            [0, 0, 0, -1, -1, -2, -1, -1, 0, 0, 0]
        ])
    convolution_result = convolution_whole_image(img, kernel)
    Laplacian_output = convolution_result_to_Laplacian_output(convolution_result, threshold)
    return Laplacian_output_to_result_image(Laplacian_output)

In [70]:
Laplace_of_Gaussian_img = Laplace_of_Gaussian(original_lena, 3000)
cv2.imwrite('result_images/Laplace_of_Gaussian_3000.bmp', Laplace_of_Gaussian_img)

True

# (e) Difference of Gaussian: 1

inhibitory sigma=3, excitatory sigma=1, kernel size 11x11

<img src = "ppt_info_images/difference_of_Gaussian.png" width = 300/>

In [71]:
def difference_of_Gaussian(img, threshold):
    kernel = np.array([
            [-1, -3, -4, -6, -7, -8, -7, -6, -4, -3, -1],
            [-3, -5, -8, -11, -13, -13, -13, -11, -8, -5, -3],
            [-4, -8, -12, -16, -17, -17, -17, -16, -12, -8, -4],
            [-6, -11, -16, -16, 0, 15, 0, -16, -16, -11, -6],
            [-7, -13, -17, 0, 85, 160, 85, 0, -17, -13, -7],
            [-8, -13, -17, 15, 160, 283, 160, 15, -17, -13, -8],
            [-7, -13, -17, 0, 85, 160, 85, 0, -17, -13, -7],
            [-6, -11, -16, -16, 0, 15, 0, -16, -16, -11, -6],
            [-4, -8, -12, -16, -17, -17, -17, -16, -12, -8, -4],
            [-3, -5, -8, -11, -13, -13, -13, -11, -8, -5, -3],
            [-1, -3, -4, -6, -7, -8, -7, -6, -4, -3, -1],
        ])
    convolution_result = convolution_whole_image(img, kernel)
    Laplacian_output = convolution_result_to_Laplacian_output(convolution_result, threshold)
    return Laplacian_output_to_result_image(Laplacian_output)

In [72]:
difference_of_Gaussian_img = difference_of_Gaussian(original_lena, 1)
cv2.imwrite('result_images/Difference_of_Gaussian_1.bmp', difference_of_Gaussian_img)

True